In [0]:
pip install newspaper3k

In [0]:
import newspaper
from newspaper import Article
import requests
from newspaper import fulltext
import pandas as pd
from json import dumps, loads, JSONEncoder, JSONDecoder
import pickle
import csv
import re
from datetime import datetime

If article url contains date, this function extracts date from the url.

In [0]:
def extract_date(url):
  date_obj = re.findall(r'/(\d{4})/(\d{1,2})/(\d{1,2})/', url)
  if date_obj:
    res = ['-'.join(tups) for tups in date_obj][0] 
    dt_date = datetime.strptime(res, '%Y-%m-%d').date()
    return dt_date.isoformat()
  return ""

Function to remove filters. 
Returns false if filters are present


In [0]:
def remove_filters(article_url, filters):
   return filters == '' or re.search(filters, article_url) is None


Function to collect all news articles and store in the list.
Function paramters are as follows:
1. url => url of news source
2. filters => regex patterns to filter. eg. 'video|opinion'

In [0]:
 def download_articles(url, filters): 
  filtered_websites = []
  news_website = newspaper.build(url, memoize_articles=False, language="en")
  df = pd.DataFrame()

  for article in news_website.articles:
    
    data_json = {}
    web_article = Article(article.url)
                
    try:
      web_article.download()
      web_article.parse()
      fulltext1 = requests.get(article.url).text
      fulltext2 = fulltext(fulltext1)
    except:
      pass

    #check keywords in body as well as article url            
    if (fulltext2.find('coronavirus') != -1 or fulltext2.find('covid-19') != -1 or fulltext2.find('sars-cov-2') != -1) or article.url.find('coronavirus') != -1 or article.url.find('covid-19') != -1 or article.url.find('sars-cov-2') != -1:
      if (remove_filters(url, filters) == True):
      #if (re.search(pattern, fulltext2) or re.search(pattern, web_article.url) ):
        #print(web_article.url, web_article.title)
      
        data_json['url'] = web_article.url                                                                                                                    
        data_json['publisher'] = web_article.source_url
        if (web_article.publish_date is None or web_article.publish_date == ""):
          print('No date')
  
          data_json['publish_date'] = extract_date(web_article.url)
        else:
          if(type(web_article.publish_date) == str):
            pass

          else:
            data_json['publish_date'] = web_article.publish_date.date().isoformat()
        remove_authors = ['May', 'Apr', 'Mar']
                  
        authors = list(web_article.authors)
              
        data_json['author'] = authors
                  
        data_json['title'] = web_article.title
        print(article.url, data_json['title'])
        data_json['image'] = web_article.top_image
        data_json['body_text'] = fulltext2
          
        filtered_websites.append(data_json)



In [0]:
filtered_websites = download_articles('https://bipartisanreport.com/', '')
print(filtered_websites)

In [0]:
df = pd.DataFrame(filtered_websites)
df.index.name = 'news_id'

df = df.drop_duplicates(subset='title')
df

# You can change the name or make it custom
df.to_csv("../dataset/news-dataset.csv")